### Clínica 2 - Entendimento de Dados & Regras de Negócio (v1)

In [1]:
# --- Imports básicos
import re, unicodedata
import math
import numpy as np
import pandas as pd
from datetime import datetime
from typing import List, Tuple

In [2]:
pd.set_option("display.max_colwidth", 200)

# --- Reprodutibilidade
RANDOM_SEED = 42
np.random.seed(RANDOM_SEED)

In [3]:
# 1) ENTENDIMENTO DE DADOS (EDA)

# LINHA DE SELEÇÃO DO INPUT  ←←← (não remover esta linha!)
INPUT_CSV = "data/dataset_clinica20252.csv"   # ajuste se necessário

# OBS: o CSV vem separado por "|"
df = pd.read_csv(INPUT_CSV, sep="|")

print("Dimensão:", df.shape)
print("Colunas:", list(df.columns))

# Tipos e amostras
display(df.dtypes)
display(df.head(3))

Dimensão: (19800, 6)
Colunas: ['cd_causa', 'cd_atendimento', 'ds_Acao_Judicial', 'ds_fatos', 'ds_Pedidos', 'ds_Qualificacao']


cd_causa            object
cd_atendimento      object
ds_Acao_Judicial    object
ds_fatos            object
ds_Pedidos          object
ds_Qualificacao     object
dtype: object

,cd_causa,cd_atendimento,ds_Acao_Judicial,ds_fatos,ds_Pedidos,ds_Qualificacao
0,CIB0500064,0825789-84.2025.8.18.0140,90 - ACAO DE REPARACAO DE DANOS,"DOS FATOS A parte Autora, pessoa idosa e hipossuficiente, é aposentada pelo INSS e percebe o valor mensal bruto de um salário mínimo. A Requerente recebe a referida quantia em seu cartão de conta ...","DOS PEDIDOS Ante ao exposto, requer: a) Sejam concedidos os benefícios da JUSTIÇA GRATUITA, nos termos da Lei n° 1.060/50, sendo certo que a autora não possui condições financeiras de arcar com as...","DOUTO JUÍZO DE DIREITO DA ___ VARA CÍVEL DA COMARCA DE TERESINA  ESTADO DO PIAUÍ. GEORGINA ALVES RABELO, brasileira, aposentada e pensionista, solteira, portadora do RG n° 1.273.995, inscrita no ..."
1,CIB0505587,1004697-72.2025.8.26.0066,90 - ACAO DE REPARACAO DE DANOS,"DOS FATOS 5. A parte autora é pessoa idosa, hipossuficiente, percebendo uma ínfima renda de seu benefício previdenciário registrado sob o n° 171.123.404-1  Pensão por Morte Previdenciaria. 6. A...","DOS PEDIDOS E REQUERIMENTOS 33. Diante do exposto, requer: a) Conceder o benefício da Justiça Gratuita, com base na lei no 1.060/50, tendo em vista a autora ser pessoa pobre e não possuir condiçõe...","(17) 99779-9177 / EXCELENTÍSSIMO SENHOR DOUTOR JUIZ DE DIREITO DA __ VARA CÍVEL DO FORO DE BARRETOS, SP. MARIA DE FATIMA BATISTA DE ARAUJO, brasileira, viúva, portadora da cédula de identidade n.o..."
2,CIB0508201,0800423-07.2025.8.15.0761,90 - ACAO DE REPARACAO DE DANOS,DOS FATOS 1. SITUAÇÃO DE VULNERABILIDADE DO CONSUMIDOR 2. DESCONTOS ILÍCITOS 1. SITUAÇÃO DE VULNERABILIDADE DO CONSUMIDOR É importante ressaltar que a parte autora não tinha conhecimento imediato ...,"DOS PEDIDOS E REQUERIMENTOS Ex Positis, requer a Vossa Excelência: a) A citação da parte promovida para que, querendo, apresente contestação no prazo legal; b) Desde já, a parte autora requer a di...","AO COLENDO JUÍZO DA VARA ÚNICA DA COMARCA DE GURINHÉM  PARAÍBA. DECLARATÓRIA DE INEXISTÊNCIA DE DÉBITOS TÍTULO DE CAPITALIZAÇÃO E SEGURO PESSOA IDOSA (Prioridade Processual) Art. 71, do Estatuto ..."


In [4]:
# Setup + utilidades

def strip_accents(s: str) -> str:
    if not isinstance(s, str): return ""
    return "".join(c for c in unicodedata.normalize("NFD", s) if unicodedata.category(c) != "Mn")

def clean_text(s: str) -> str:
    s = (s or "").replace("\n", " ")
    return re.sub(r"\s+", " ", s).strip()

# Constantes
UF_LIST = set("AC AL AP AM BA CE DF ES GO MA MT MS MG PA PB PR PE PI RJ RN RS RO RR SC SP SE TO".split())
MESES_PT = {"janeiro":1,"fevereiro":2,"março":3,"marco":3,"abril":4,"maio":5,"junho":6,"julho":7,"agosto":8,"setembro":9,"outubro":10,"novembro":11,"dezembro":12}


In [9]:
# lê as colunas textuais, normaliza e filtra casos cujo texto contenha "crédito consignado"
# salva output.xlsx com todas as colunas
def filtra_credito_consignado(df: pd.DataFrame) -> pd.DataFrame:
    """
    Procura 'crédito consignado' nas colunas textuais e salva output.xlsx
    contendo todas as colunas.
    """
    #
    text_cols = ["ds_Acao_Judicial", "ds_fatos", "ds_Pedidos", "ds_Qualificacao"]
    present = [c for c in text_cols if c in df.columns]
    txt = df[present].astype(str).agg(" ".join, axis=1).map(lambda x: strip_accents(x).lower())
    mask = txt.str.contains("credito consignado", na=False)

    out = df.loc[mask].copy()
    out.to_excel("output/output.xlsx", index=False)
    return out

def clean_illegal_chars(df):
    # Expressão regular para remover caracteres ilegais do Excel
    return df.map(lambda x: re.sub(r'[\x00-\x1f\x7f-\x9f]', '', str(x)) if isinstance(x, str) else x)



In [ ]:
filtro = clean_illegal_chars(df)

filtro = filtra_credito_consignado(filtro)   # salva output.xlsx (todas as colunas)
filtro.head()

,cd_causa,cd_atendimento,ds_Acao_Judicial,ds_fatos,ds_Pedidos,ds_Qualificacao
99,CIB0532943,0801915-71.2024.8.10.0126,90 - ACAO DE REPARACAO DE DANOS,"DOS FATOS Trata-se de uma ação declaratória de nulidade contratual em que se originou quando a parte requerente (idosa, aposentada e hipossuficiente de conhecimentos técnicos) ao analisar o Extrat...","DOS PEDIDOS Diante do exposto, requer: a) A juntada aos autos do protocolo administrativo mencionado; b) O regular prosseguimento do feito, com o reconhecimento do interesse de agir da parte autor...",Num. 149459114 - Pág. 1 Assinado eletronicamente por: CAIO GUILHERME MIRANDA DE SOUSA - 22/05/2025 15:13:36 Número do documento: 25052215133663500000138721856 SENACON Secretaria Nacional do Consum...
396,CIB0504651,0137910-20.2025.8.04.1000,90 - ACAO DE REPARACAO DE DANOS,"DOS FATOS A parte Autora, Sra. Christiane Araujo de Paula, servidora pública municipal, firmou com o Banco Bradesco S.A., em 05 de dezembro de 2019, contrato de empréstimo consignado em folha de p...","DOS PEDIDOS Diante do exposto, requer-se: 1) seja implantado o JUÍZO 100% DIGITAL neste feito, na forma da Resolução CNJ no 345/2020; 2) a concessão da inversão do ônus da prova, nos termos do art...","11 99909-5678 EXCELENTÍSSIMO SENHOR DOUTOR JUIZ DE DIREITO DO FORO DA COMARCA CENTRAL DA REGIÃO METROPOLITANA DE MANAUS ESTADO DO AMAZONAS JUÍZO 100% DIGITAL CHRISTIANE ARAÚJO DE PAULA, brasileir..."
2413,CIB0479143,0104620-14.2025.8.04.1000,90 - ACAO DE REPARACAO DE DANOS,"DOS FATOS A parte autora é cliente do Banco Bradesco S/A, ora requerido, sob a conta corrente n° 76270-9, na agência no 3726. Acontece que, o requerido sem que houvesse qualquer manifestação expre...","DOS PEDIDOS Diante do exposto, requer-se: a) Conceda Vossa Excelência a Requerente, o benefício da GRATUIDADE DA JUSTIÇA conforme o art. 5o, inciso LXXIV da CRFB e art. 98 e ss. Do CPC/15; b) Deix...","AO JUÍZO DE DIREITO DA VARA CÍVEL DA COMARCA DE MANAUS AMAZONAS. SOLICITA PRIORIDADE NA TRAMITAÇÃO DO PROCESSO A AUTORA DA AÇÃO É IDOSA NA FORMA DA LEI. (ART. 1.048, INC. I DO CPC C/C ESTATUTO D..."
2732,CIB0460874,0816256-14.2025.8.20.5001,90 - ACAO DE REPARACAO DE DANOS,"DOS FATOS A parte autora é beneficiária do INSS e conta atualmente 49 anos de idade, pensionista, e sobrevive exclusivamente do seu benefício previdenciário na quantia de R$ 1.412,00 reais. Vale d...",DOS PEDIDOS Num. 145799694 - Pág. 14 Pág. Total - 14 Assinado eletronicamente por: ANELIZA GURGEL DE MEDEIROS - 18/03/2025 20:08:28 Número do documento: 25031820082847800000135941344 Ante o expost...,"AO JUIZO DE UMA DAS VARAS CÍVEIS DA COMARCA DE NATAL/RN, A QUEM COUBER POR DISTRIBUIÇÃO LEGAL; MARIA ISABEL GOMES DA SILVA, brasileira, aposentada, inscrita no CPF sob o n. 009.571.444-92, residen..."
2814,CIB0465386,1000682-46.2025.8.26.0103,90 - ACAO DE REPARACAO DE DANOS,"DOS FATOS Primeiramente, a parte Requerente sempre trabalhou arduamente, arcando com suas contas e obrigações. Fato que, em um momento de necessidade buscou a Requerida para contrair um Empréstimo...","requer: Para conferir o original, acesse o site informe o processo 1000682-46.2025.8.26.0103 e código 8vRm63h8. 29 a) Determinar a citação do Banco Requerido, nos termos do art. 246, I, do CPC/20...","EXCELENTÍSSIMO SENHOR DOUTOR JUIZ DE DIREITO DA VARA CÍVEL DA COMARCA DE CACONDE ESTADO DE SÃO PAULO. TOCHIHARU SATO, brasileiro, casado, beneficiário do LOAS, titular do registro único sob no 52..."


In [12]:
filtro.to_excel("output/output1.xlsx", index=False)

In [13]:
filtro.describe()

,cd_causa,cd_atendimento,ds_Acao_Judicial,ds_fatos,ds_Pedidos,ds_Qualificacao
count,481,481,481,481,481,481
unique,481,481,3,481,481,448
top,CIB0532943,0801915-71.2024.8.10.0126,INDENIZATÓRIA,"DOS FATOS Trata-se de uma ação declaratória de nulidade contratual em que se originou quando a parte requerente (idosa, aposentada e hipossuficiente de conhecimentos técnicos) ao analisar o Extrat...","DOS PEDIDOS Diante do exposto, requer: a) A juntada aos autos do protocolo administrativo mencionado; b) O regular prosseguimento do feito, com o reconhecimento do interesse de agir da parte autor...","EXCELENTÍSSIMO SENHOR DOUTOR JUIZ DE DIREITO DA ___ VARA CÍVEL DA COMARCA DE JOSÉ DE FREITAS, ESTADO DO PIAUÍ. Procedimento comum PRIORIDADE PROCESSUAL - (art. 1.211-A do CPC) TERESINHA DE JESUS C..."
freq,1,1,450,1,1,6


A função `filtra_credito_consignado` gera uma saída `output.xlsx` apenas com a coluna `cd_atendimento` dos casos de **crédito consignado**.

**O que a função faz:**

1 - Junta os textos das colunas `ds_Acao_Judicial`, `ds_fatos`, `ds_Pedidos` e `ds_Qualificacao`.

2 - Remove acentos e coloca tudo em minúsculas.

3 - Filtra linhas que contêm "crédito consignado".

4 - Retorna um novo DataFrame com `cd_atendimento` dessas linhas.

5 - Salva no excel.

In [41]:
# Funções auxiliares para extração

def extract_cnpjs(text: str) -> str:
    text = clean_text(text)
    raw = re.findall(r"\b\d{2}\.?\d{3}\.?\d{3}/?\d{4}-?\d{2}\b", text)
    nums = [re.sub(r"\D", "", c) for c in raw if len(re.sub(r"\D", "", c)) == 14]
    nums = sorted(set(nums))
    return ",".join(nums) if nums else "vazio"

def extract_valor_causa(text: str) -> float:
    text = clean_text(text)
    matches = re.findall(r"(?:R\$\s*)?(\d{1,3}(?:\.\d{3})*,\d{2}|\d+,\d{2})", text)
    valores = [float(v.replace(".", "").replace(",", ".")) for v in matches]
    return max(valores) if valores else 0.0

def extract_dt_distribuicao(text: str) -> str:
    t = strip_accents(clean_text(text)).lower()
    for pattern in [
        r"\b(20\d{2})-(0[1-9]|1[0-2])-(0[1-9]|[12]\d|3[01])\b",   # ISO
        r"\b(0?[1-9]|[12]\d|3[01])[/\-](0?[1-9]|1[0-2])[/\-](20\d{2})\b", # dd/mm/yyyy
    ]:
        m = re.search(pattern, t)
        if m:
            parts = m.groups()
            if len(parts) == 3 and parts[0].startswith("20"):
                y, mm, dd = parts[0], parts[1], parts[2]
            else:
                dd, mm, y = parts[0], parts[1], parts[2]
            return f"{y}-{mm.zfill(2)}-{dd.zfill(2)}"
    return ""

def classify_tipo_vara(text: str) -> str:
    t = strip_accents(clean_text(text)).lower()
    if "juizado especial" in t or "jecc" in t:
        return "JE"
    return "G1"

def extract_uf(text: str) -> str:
    tokens = re.findall(r"\b[A-Z]{2}\b", text.upper())
    for tk in tokens:
        if tk in UF_LIST:
            return tk
    return ""


In [42]:
# Extrair nome de empresas

def extract_nome_empresa(text: str) -> str:
    """
    Captura nomes de empresas no polo passivo.
    Ex: BANCO DO BRASIL S.A., BANCO SANTANDER S/A, BRADESCO, etc.
    """
    t = clean_text(text).upper()
    m = re.search(r"(BANCO [A-Z ]{2,}(?:S\.?A\.?|S\/A|LTDA|EIRELI|ME)?)", t)
    if m:
        return m.group(1).strip(" ,;")
    m = re.search(r"([A-Z ]{2,}(?:S\.?A\.?|S\/A|LTDA|EIRELI|ME))", t)
    if m:
        return m.group(1).strip(" ,;")
    return "vazio"


In [43]:
# ---------------- CNPJ: validação e parsing ----------------
def cnpj_is_valid(cnpj_digits: str) -> bool:
    """Valida dígitos verificadores do CNPJ (14 dígitos)."""
    if len(cnpj_digits) != 14 or len(set(cnpj_digits)) == 1:
        return False
    nums = [int(x) for x in cnpj_digits]
    for i in [12, 13]:
        if i == 12: pesos = [5,4,3,2,9,8,7,6,5,4,3,2]
        else:       pesos = [6,5,4,3,2,9,8,7,6,5,4,3,2]
        soma = sum(a*b for a,b in zip(nums[:i], pesos))
        dig = 11 - (soma % 11)
        dig = 0 if dig >= 10 else dig
        if nums[i] != dig: 
            return False
    return True

def find_cnpjs_pos(text: str) -> List[Tuple[str,int]]:
    """Encontra CNPJs válidos e suas posições no texto."""
    out = []
    for m in re.finditer(r"\b\d{2}\.?\d{3}\.?\d{3}/?\d{4}-?\d{2}\b", text):
        digits = re.sub(r"\D", "", m.group(0))
        if len(digits) == 14 and cnpj_is_valid(digits):
            out.append((digits, m.start()))
    return out

# ---------------- Empresa: nome e span ----------------
_COMPANY_SUFFIX = r"(?:S\.?A\.?|S\/A|LTDA|EIRELI|ME)"
def find_company_spans(text: str) -> List[Tuple[str,int,int]]:
    """
    Retorna [(nome, start, end), ...] para empresas típicas no polo passivo.
    Captura 'BANCO ...', ou qualquer bloco com sufixo societário.
    """
    T = clean_text(text).upper()
    spans = []
    # 1) BANCO ...
    for m in re.finditer(r"(BANCO(?: [A-Z0-9&'\.\-]{2,}){1,8})", T):
        name = m.group(1).strip(" ,;")
        spans.append((name, m.start(1), m.end(1)))
    # 2) QUALQUER NOME COM SUFIXO SOCIETÁRIO
    for m in re.finditer(rf"([A-Z0-9][A-Z0-9 \.&'\-]{{2,}}{_COMPANY_SUFFIX})", T):
        name = m.group(1).strip(" ,;")
        spans.append((name, m.start(1), m.end(1)))
    # dedup por nome
    seen, uniq = set(), []
    for n,s,e in spans:
        if n not in seen:
            uniq.append((n,s,e))
            seen.add(n)
    return uniq

def pick_company_and_cnpjs(text: str, win_after=200, win_before=100) -> Tuple[str, List[str]]:
    """
    Escolhe UM nome de empresa e os CNPJs que estão próximos a esse nome.
    Regra: prioriza o primeiro span; se sem CNPJ no entorno, tenta o próximo; 
    se nada, usa primeiro CNPJ válido da qualificação.
    """
    T = clean_text(text)
    companies = find_company_spans(T)
    cnpjs_pos = find_cnpjs_pos(T)

    # helper: cnpjs próximos a um span
    def cnpjs_near(start, end):
        near = []
        for cnpj, pos in cnpjs_pos:
            if (start - win_before) <= pos <= (end + win_after):
                near.append(cnpj)
        return sorted(set(near))

    # tenta associar por proximidade
    for name, s, e in companies:
        near = cnpjs_near(s, e)
        if near:
            return name, near

    # fallback: sem nome confiável, usa primeiro CNPJ válido (se houver)
    if cnpjs_pos:
        return "vazio", [cnpjs_pos[0][0]]

    return "vazio", []

# ---------------- Outros campos (ajustes leves) ----------------
def extract_valor_causa_priorizando_label(text: str) -> float:
    """
    Procura primeiro números próximos a 'valor da causa' (janela curta).
    Se não achar, cai no maior valor do texto.
    """
    T = clean_text(text)
    # janela +/- 80 chars ao redor de 'valor da causa'
    for m in re.finditer(r"valor (?:da|de) causa", strip_accents(T).lower()):
        a, b = max(0, m.start()-80), m.end()+80
        trecho = T[a:b]
        nums = re.findall(r"(?:R\$\s*)?(\d{1,3}(?:\.\d{3})*,\d{2}|\d+,\d{2})", trecho)
        if nums:
            vals = [float(v.replace(".","").replace(",", ".")) for v in nums]
            return max(vals)
    # fallback: maior do texto
    nums = re.findall(r"(?:R\$\s*)?(\d{1,3}(?:\.\d{3})*,\d{2}|\d+,\d{2})", T)
    vals = [float(v.replace(".","").replace(",", ".")) for v in nums]
    return max(vals) if vals else 0.0

def extract_dt_distribuicao(text: str) -> str:
    t = strip_accents(clean_text(text)).lower()
    m = re.search(r"\b(20\d{2})-(0[1-9]|1[0-2])-(0[1-9]|[12]\d|3[01])\b", t)
    if m: return f"{m.group(1)}-{m.group(2)}-{m.group(3)}"
    m = re.search(r"\b(0?[1-9]|[12]\d|3[01])[/\-](0?[1-9]|1[0-2])[/\-](20\d{2})\b", t)
    if m:
        dd, mm, y = int(m.group(1)), int(m.group(2)), int(m.group(3))
        return f"{y:04d}-{mm:02d}-{dd:02d}"
    return ""

def classify_tipo_vara(text: str) -> str:
    t = strip_accents(clean_text(text)).lower()
    return "JE" if ("juizado especial" in t or "jecc" in t) else "G1"

def extract_uf(text: str) -> str:
    for tok in re.findall(r"\b[A-Z]{2}\b", clean_text(text).upper()):
        if tok in UF_LIST:
            return tok
    return ""


In [44]:
def criar_colunas(df: pd.DataFrame) -> pd.DataFrame:
    """
    Usa principalmente a 'ds_Qualificacao' (onde normalmente aparece o polo passivo),
    mas consulta 'ds_fatos' e 'ds_Pedidos' para valor e data.
    """
    qual = df.get("ds_Qualificacao", "").astype(str)
    fatos = df.get("ds_fatos", "").astype(str)
    pedidos = df.get("ds_Pedidos", "").astype(str)

    # nome + cnpjs amarrados por proximidade
    pares = qual.map(lambda t: pick_company_and_cnpjs(t))
    nome_empresa = pares.map(lambda x: x[0])
    cnpjs = pares.map(lambda x: ",".join(x[1]) if x[1] else "vazio")

    texto_valor = (fatos + " " + pedidos).astype(str)
    texto_data  = (pedidos + " " + qual).astype(str)

    out = pd.DataFrame({
        "cd_atendimento": df["cd_atendimento"].astype(str),
        "nome_empresa": nome_empresa,
        "cnpj": cnpjs,
        "valor_causa": texto_valor.map(extract_valor_causa_priorizando_label),
        "dt_distribuicao": texto_data.map(extract_dt_distribuicao),
        "tipo_vara": qual.map(classify_tipo_vara),
        "uf": qual.map(extract_uf),
    })

    out.to_excel("output/output_colunas.xlsx", index=False)
    return out


In [ ]:
resultado = criar_colunas(df)
display(resultado.head(10))

,cd_atendimento,nome_empresa,cnpj,valor_causa,dt_distribuicao,tipo_vara,uf
0,0825789-84.2025.8.18.0140,BANCO BRADESCO FINANCIAMENTOS,07207996000150,21906.64,2025-05-14,G1,PI
1,1004697-72.2025.8.26.0066,BANCO BRADESCO S.A,60746948000112,10000.00,,G1,SP
2,0800423-07.2025.8.15.0761,BANCO DO BRADESCO S.A,60746948000112,11409.08,2025-05-22,G1,PB
3,1004875-69.2025.8.26.0438,BANCO BRADESCO,60746948000112,11256.04,,G1,SP
4,0010630-50.2025.8.27.2706,BANCO BRADESCO S.A.,60746948000112,15215.42,,G1,TO
5,0092601-36.2025.8.05.0001,vazio,vazio,15000.00,,G1,
6,0801575-51.2025.8.18.0068,BANCO BRADESCO S.A.,60746948000112,5159.84,2025-06-02,G1,PI
7,0801452-38.2025.8.18.0073,E BRADESCO SEGUROS S/A,33055146004776,0.00,2017-02-21,G1,PI
8,0800895-81.2025.8.10.0038,vazio,vazio,0.00,2025-05-19,G1,
9,0148716-17.2025.8.04.1000,BANCO BRADESCO S.A INSCRITO NO CNPJ 60.746.948,60746948000112,16804.00,,G1,AM


### Entrega 6 – Criação de Novas Colunas

Nesta etapa, foram criadas novas colunas a partir dos textos jurídicos para facilitar as análises:
- **cnpj:** identifica a empresa ré no processo.
- **valor_causa:** extrai o valor monetário da causa.
- **dt_distribuicao:** data da petição (início do processo).
- **tipo_vara:** classifica entre Justiça Comum (G1) e Juizado Especial (JE).
- **uf:** sigla do estado.
- **nome_empresa:** tenta capturar o nome da empresa envolvida (p.ex. Banco do Brasil S.A.).

Essas colunas são extraídas automaticamente com expressões regulares e regras textuais.
O arquivo final é salvo como `output_colunas.xlsx`.


### `output.xlsx`

| Coluna            | Descrição                                                               |
| ----------------- | ----------------------------------------------------------------------- |
| `cd_atendimento`  | Identificador do processo (igual à coluna original).                    |
| `nome_empresa`    | Nome da empresa ré (extraído do texto da petição, polo passivo).        |
| `cnpj`            | Todos os CNPJs encontrados (somente 14 dígitos, separados por vírgula). |
| `valor_causa`     | Valor total da causa em formato numérico (R$ 1.234,56 → 1234.56).       |
| `dt_distribuicao` | Data da petição no formato `YYYY-MM-DD`.                                |
| `tipo_vara`       | “JE” (Juizado Especial) ou “G1” (Justiça Comum).                        |
| `uf`              | Sigla do estado (SP, BA, PI...).                                        |
